In [ ]:
# ------------------------------------------------

import os

#source /etc/os-release && echo $PRETTY_NAME
with open('/etc/os-release') as fh:
  for line in fh:
    if line and line[0] != '#':
      os.environ[line.split('=')[0]] = line.split('=')[1]
print(f'Running {os.environ["PRETTY_NAME"]}') # on {os.environ["HOST"]} as {os.environ["USER"]}')

# ------------------------------------------------

from IPython.display import display, clear_output
import time, sys
tick = time.time()
# Install mamba without resetting the kernel alla condacolab
#!wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh
!bash Mambaforge-Linux-x86_64.sh -bfp /usr/local
sys.path.append('/usr/local/lib/python3.10/site-packages')
!mamba config --set auto_update_conda false
!pip -q install rdkit
!pip -q install pdb2pqr
!pip install -q git+https://github.com/matteoferla/DTC-compchem-practical.git
!mamba install -y -c bioconda autodock-vina
!mamba conda install -y -c conda-forge oddt openbabel
tock = time.time()
clear_output()
print(f'Installation time: {tock - tick}')